In [1]:
import pandas as pd
from decimal import Decimal, getcontext
from shapely import affinity
from shapely.geometry import Polygon
from shapely.ops import unary_union

getcontext().prec = 30
scale_factor = 1

class ChristmasTree:
    def __init__(self, center_x='0', center_y='0', angle='0'):
        self.center_x = Decimal(center_x)
        self.center_y = Decimal(center_y)
        self.angle = Decimal(angle)
        trunk_w = Decimal('0.15')
        trunk_h = Decimal('0.2')
        base_w = Decimal('0.7')
        mid_w = Decimal('0.4')
        top_w = Decimal('0.25')
        tip_y = Decimal('0.8')
        tier_1_y = Decimal('0.5')
        tier_2_y = Decimal('0.25')
        base_y = Decimal('0.0')
        trunk_bottom_y = -trunk_h
        initial_polygon = Polygon([
            (Decimal('0.0') * scale_factor, tip_y * scale_factor),
            (top_w / Decimal('2') * scale_factor, tier_1_y * scale_factor),
            (top_w / Decimal('4') * scale_factor, tier_1_y * scale_factor),
            (mid_w / Decimal('2') * scale_factor, tier_2_y * scale_factor),
            (mid_w / Decimal('4') * scale_factor, tier_2_y * scale_factor),
            (base_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, base_y * scale_factor),
            (trunk_w / Decimal('2') * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, trunk_bottom_y * scale_factor),
            (-(trunk_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(base_w / Decimal('2')) * scale_factor, base_y * scale_factor),
            (-(mid_w / Decimal('4')) * scale_factor, tier_2_y * scale_factor),
            (-(mid_w / Decimal('2')) * scale_factor, tier_2_y * scale_factor),
            (-(top_w / Decimal('4')) * scale_factor, tier_1_y * scale_factor),
            (-(top_w / Decimal('2')) * scale_factor, tier_1_y * scale_factor),
        ])
        rotated = affinity.rotate(initial_polygon, float(self.angle), origin=(0, 0))
        self.polygon = affinity.translate(rotated, xoff=float(self.center_x * scale_factor), yoff=float(self.center_y * scale_factor))

def get_tree_list_side_length(tree_list):
    all_polygons = [t.polygon for t in tree_list]
    bounds = unary_union(all_polygons).bounds
    return Decimal(max(bounds[2] - bounds[0], bounds[3] - bounds[1])) / scale_factor

def get_total_score(dict_of_side_length):
    score = Decimal('0')
    for k, v in dict_of_side_length.items():
        score += v ** 2 / Decimal(str(k))
    return score

def parse_csv(csv_path):
    result = pd.read_csv(csv_path)
    result['x'] = result['x'].astype(str).str.strip().str.lstrip('s')
    result['y'] = result['y'].astype(str).str.strip().str.lstrip('s')
    result['deg'] = result['deg'].astype(str).str.strip().str.lstrip('s')
    result[['group_id', 'item_id']] = result['id'].str.split('_', n=2, expand=True)
    dict_of_tree_list = {}
    dict_of_side_length = {}
    for group_id, group_data in result.groupby('group_id'):
        tree_list = [ChristmasTree(center_x=row['x'], center_y=row['y'], angle=row['deg']) for _, row in group_data.iterrows()]
        dict_of_tree_list[group_id] = tree_list
        dict_of_side_length[group_id] = get_tree_list_side_length(tree_list)
    return dict_of_tree_list, dict_of_side_length

print('Scoring santa-2025.csv (saspav dataset)...')
dict_trees, dict_sides = parse_csv('santa-2025.csv')
score = get_total_score(dict_sides)
print(f'Score: {float(score):.6f}')
print(f'Target: 68.919154')
print(f'Gap: {float(score) - 68.919154:.6f}')

Scoring santa-2025.csv (saspav dataset)...


Score: 70.658891
Target: 68.919154
Gap: 1.739737


In [ ]:
print('Scoring submission.csv (bucket-of-chump dataset)...')
dict_trees2, dict_sides2 = parse_csv('submission.csv')
score2 = get_total_score(dict_sides2)
print(f'Score: {float(score2):.6f}')
print(f'Target: 68.919154')
print(f'Gap: {float(score2) - 68.919154:.6f}')